In [1]:
import tensorflow as tf
from tensorflow import keras
import _my_tools as mt
import numpy as np
import os

def compareRes(i,folder=None):
    mt.showImg(str(i).zfill(2),X_test[i,:,:,0],y_test[i,:,:,0],res[i,:,:,0],X_test[i,:,:,1],folder=folder)

ModuleNotFoundError: ignored

In [0]:
X_train, y_train, X_test, y_test = mt.loadDataFloat("dataset_interpolation/")

X_train = X_train.swapaxes(1,3)
X_train = X_train.swapaxes(1,2)
X_test = X_test.swapaxes(1,3)
X_test = X_test.swapaxes(1,2)

y_train = np.expand_dims(y_train,3)
y_test = np.expand_dims(y_test,3)

### CNN model 1
The simplest autoencoder - autodecoder net.

In [0]:
input_img = keras.layers.Input((64,64,2))

m = keras.layers.Conv2D(8,(3,3),activation='relu',padding='same')(input_img)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(16,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(m)
encoded = keras.layers.MaxPooling2D((2,2),padding='same')(m)

m = keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoded)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(16,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(8,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(4,(3,3),activation='relu',padding='same')(m)

m = keras.layers.UpSampling2D((2,2))(m)
decoded = keras.layers.Conv2D(1,(3,3),activation='relu',padding='same')(m)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 2)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 8)         152       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 32)          0         
__________

In [0]:
checkpoint_path = "07/cp2"
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             save_weights_only=True,
                                             verbose=1)

batch_size = 8
num_epoch = 1
#model training
model_log = autoencoder.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,          
          validation_data=(X_test, y_test),
          callbacks = [cp_callback])

Train on 63324 samples, validate on 15831 samples
Epoch 1/1
63312/63324 [============================>.] - ETA: 0s - loss: 1.8349
Epoch 00001: saving model to 07/cp2

Consider using a TensorFlow optimizer from `tf.train`.
63324/63324 [==============================] - 350s 6ms/step - loss: 1.8348 - val_loss: 1.7093


In [0]:
autoencoder.load_weights("07/cp2")

In [0]:
res = autoencoder.predict(X_test[:50])

for i in range(30):
    compareRes(i,folder=None)

### CNN model 2
Same as before with a fully connected layer at the end.

In [0]:
input_img = keras.layers.Input((64,64,2))

m = keras.layers.Conv2D(8,(3,3),activation='relu',padding='same')(input_img)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(16,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(m)
m = keras.layers.MaxPooling2D((2,2),padding='same')(m)
m = keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(m)
encoded = keras.layers.MaxPooling2D((2,2),padding='same')(m)

m = keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoded)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(16,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(8,(3,3),activation='relu',padding='same')(m)
m = keras.layers.UpSampling2D((2,2))(m)
m = keras.layers.Conv2D(4,(3,3),activation='relu',padding='same')(m)

m = keras.layers.UpSampling2D((2,2))(m)
decoded = keras.layers.Conv2D(1,(3,3),activation='relu',padding='same')(m)

a = keras.layers.Flatten()(decoded)
a = keras.layers.Dense(4096,activation='sigmoid')(a)
a = keras.layers.Reshape((64,64,1))(a)


autoencoder = keras.Model(input_img, a)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 64, 64, 2)         0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 64, 64, 8)         152       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 32, 32, 16)        1168      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 8, 8, 32)          0         
__________

In [0]:
checkpoint_path = "07/cp4"
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             save_weights_only=True,
                                             verbose=1)

batch_size = 8
num_epoch = 1
#model training
model_log = autoencoder.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,          
          validation_data=(X_test, y_test),
          callbacks = [cp_callback])

Train on 63324 samples, validate on 15831 samples
Epoch 1/1
63320/63324 [============================>.] - ETA: 0s - loss: 1.7245
Epoch 00001: saving model to 07/cp4

Consider using a TensorFlow optimizer from `tf.train`.
63324/63324 [==============================] - 2337s 37ms/step - loss: 1.7245 - val_loss: 1.7118


In [0]:
 autoencoder.load_weights("07/cp4")

In [0]:
res = autoencoder.predict(X_test[0:30])

for i in range(30):
    compareRes(i,folder=None)